# Finding coordinates of places of worship

The first thing we need to do is download the coordinates of potential church buildings from OpenStreetMap.

In [1]:
import requests
import json
import pandas as pd
import numpy as np

We make a query to the Overpass API using an arbitrary bounding box, which goes from Aldham in the west to Elmstead in the east, and Horkesley Heath in the north to Fingringhoe in the south.

In [2]:
request = """
[out:json][timeout:25][bbox:51.844,0.778,51.942, 1.015];
// gather results
(
  // query part for: “church”
  node["amenity"="place_of_worship"]["religion"="christian"]["building"!="church"];
way["amenity"="place_of_worship"]["religion"="christian"]["building"!="church"];
  relation["amenity"="place_of_worship"]["religion"="christian"]["building"!="church"];
);
// print results

out body;
>;
out skel qt;
"""

overpass_url = "http://overpass-api.de/api/interpreter"
response = requests.get(overpass_url, params={'data': request})

## Checking what we've got to work with

We can see the bounded area in the image below. We get 591 entities back from OpenStreetMap. 548 of these are nodes that are either point descriptions of a place of worship, or constituent parts of one of 43 "ways": an OpenStreetMap term used to described features like roads or (as in this case) the outline of buildings.

50 of the returns have names, so these are probably specific places of worship. Only 24 returns have any ```building``` information: one gospel hall building, one converted residential building, and 22 counts of "yes, this is a building".

![Map of Colchester showing the points returned from the OpenStreetMap query in blue-ringed markers](writeup_files/OSM_bbox_view.png)

The code below creates a dataframe with what OpenStreetMap's API sent us and rummages through it in various ways.

In [3]:
df = pd.json_normalize(response.json()['elements'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     591 non-null    object 
 1   id                       591 non-null    int64  
 2   lat                      548 non-null    float64
 3   lon                      548 non-null    float64
 4   tags.amenity             50 non-null     object 
 5   tags.denomination        43 non-null     object 
 6   tags.name                50 non-null     object 
 7   tags.religion            50 non-null     object 
 8   tags.source              6 non-null      object 
 9   tags.wikidata            24 non-null     object 
 10  tags.addr:city           11 non-null     object 
 11  tags.addr:housename      4 non-null      object 
 12  tags.addr:postcode       14 non-null     object 
 13  tags.addr:street         13 non-null     object 
 14  tags.website             2

In [4]:
df['tags.building'].value_counts()

yes            22
gospel_hall     1
residential     1
Name: tags.building, dtype: int64

In [5]:
df['type'].value_counts()

node    548
way      43
Name: type, dtype: int64

In [6]:
df.sample(20)

,type,id,lat,lon,tags.amenity,tags.denomination,tags.name,tags.religion,tags.source,tags.wikidata,...,tags.source:addr,tags.source:alt_name,tags.phone,tags.wheelchair,tags.toilets:wheelchair,tags.source:building,tags.addr:housenumber,tags.opening_hours,tags.layer,tags.level
222,node,1637154021,51.872380,0.871695,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,node,1617190931,51.889566,0.863678,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566,node,725493684,51.926527,0.985571,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,node,1617190995,51.889561,0.863170,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,node,1638640939,51.859007,0.959159,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,node,725493712,51.926612,0.986016,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,node,1577445316,51.898208,0.785804,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,way,150838753,NaN,NaN,place_of_worship,anglican,"St Barnabas, Old Heath",christian,NaN,Q105083493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,node,2476897947,51.851480,0.829909,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,node,1617190983,51.889541,0.863320,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Giving places defined by an OpenStreetMap ```way``` a single pair of coordinates

Most of the places of worship are defined as building outlines rather than a single point on the map. This is good! But it does mean that we'd end up making several requests to Google for each building, which isn't good for speed, cost or storage. First we create a separate dataframe of the coordinates of each node that is used to make up a ```way```.

In [7]:
nodes_from_ways = df[(df['type']=='node') & (df['tags.amenity']!="place_of_worship")].dropna(axis=1).set_index('id',drop=True)
nodes_from_ways.head()

,type,lat,lon
id,,,
2476897942,node,51.851449,0.829922
2476897943,node,51.851484,0.830257
2476897944,node,51.851488,0.829958
2476897945,node,51.851554,0.830227
2476897946,node,51.851508,0.829950


Now we can go through the nodes in a place of worship's ```way``` and convert those into coordinates. If we average them, we'll get a coordinate near the centre of the building. In theory we should take the projection system into account, but we are dealing with tiny differences here so let's not overcomplicate things.

In [8]:
def find_centre(nodes, nodes_df):
    lats = []
    lons = []
    for node in nodes:
        lats.append(nodes_df.loc[int(node),'lat'])
        lons.append(nodes_df.loc[int(node),'lon'])
    mean_lat = sum(lats) / len(lats)
    mean_lon = sum(lons) / len(lons)
    
    return mean_lat.round(6), mean_lon.round(6)

Give it a test...

In [9]:
find_centre([2476897942], nodes_from_ways)

(51.851449, 0.829922)

Perfect. We'll create a dataframe of all the places of worship defined by ```way```s...

In [10]:
centre_guesses = df[df['nodes'].notna()]['nodes'].apply(lambda x: find_centre(x, nodes_from_ways))
centres_df = pd.DataFrame(centre_guesses)
centres_df[['lat','lon']] = centres_df['nodes'].to_list()
centres_df.head()

,nodes,lat,lon
7,"(51.89002, 0.896746)",51.890020,0.896746
8,"(51.883674, 0.817888)",51.883674,0.817888
9,"(51.894388, 0.999748)",51.894388,0.999748
10,"(51.926554, 0.985736)",51.926554,0.985736
11,"(51.855057, 0.959371)",51.855057,0.959371


...and then stick them on to the rest of the data.

In [11]:
full_df = df.join(centres_df, rsuffix='_guess')

In [12]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     591 non-null    object 
 1   id                       591 non-null    int64  
 2   lat                      548 non-null    float64
 3   lon                      548 non-null    float64
 4   tags.amenity             50 non-null     object 
 5   tags.denomination        43 non-null     object 
 6   tags.name                50 non-null     object 
 7   tags.religion            50 non-null     object 
 8   tags.source              6 non-null      object 
 9   tags.wikidata            24 non-null     object 
 10  tags.addr:city           11 non-null     object 
 11  tags.addr:housename      4 non-null      object 
 12  tags.addr:postcode       14 non-null     object 
 13  tags.addr:street         13 non-null     object 
 14  tags.website             2

We don't want to be faffing with two different pairs of coordinate columns, so here we smash them together and create a ```coords_imputed``` column to let us trace which ones have had the averaging process applied. We also drop anything that isn't a place of worship in and of itself – we don't want to keep all the individual nodes that constituted ```way```s – by checking the ```tags.amenity``` column.

In [13]:
full_df.apply(lambda row: row['lat'] if row['lat'] != np.nan else row['lat_guess'], axis=1)
places_of_worship = full_df.dropna(subset='tags.amenity').copy()

In [14]:
places_of_worship['lat'] = places_of_worship.apply(lambda row: row['lat_guess'] if np.isnan(row['lat']) else row['lat'], axis=1)
places_of_worship['lon'] = places_of_worship.apply(lambda row: row['lon_guess'] if np.isnan(row['lon']) else row['lon'], axis=1)
places_of_worship['coords_imputed'] = places_of_worship['nodes'].notna()

In [15]:
places_of_worship[['tags.name','tags.denomination','type','lat','lon','nodes','coords_imputed']].sample(20)

,tags.name,tags.denomination,type,lat,lon,nodes,coords_imputed
24,"St Matthew, Colchester",anglican,way,51.896684,0.934167,"[1635864731, 1635864728, 1635864729, 163586472...",True
35,"St Marys, Little Birch (ruin)",NaN,way,51.851490,0.830044,"[2476897945, 2476897946, 2476897944, 247689794...",True
30,St Monica Roman Catholic Church,catholic,way,51.859073,0.959094,"[1638640938, 1638640940, 1638640941, 163864093...",True
14,"St Lawrence, East Donyland",anglican,way,51.855661,0.946258,"[280846218, 1584765682, 1584765677, 1584765674...",True
39,The Salvation Army Family Worship and Communit...,salvation_army,way,51.886494,0.895378,"[3982961224, 3982961221, 3982961204, 398296120...",True
45,Seventh Day Adventist Church,seventh_day_adventist,way,51.895173,0.895006,"[4999912445, 4999912446, 4999912447, 499991244...",True
34,Colchester Gospel Hall,gospel,way,51.882990,0.913232,"[2006157589, 2006157583, 2006157585, 200615759...",True
15,"St Michael and All Angels, Copford",anglican,way,51.869317,0.809385,"[1588968353, 1588968246, 1588968359, 158896835...",True
8,St. Albright's Church,anglican,way,51.883674,0.817888,"[412182280, 412182283, 1613783473, 1613783464,...",True
42,Salvation Army Colchester Mount Zion Corps,salvation_army,way,51.905430,0.923309,"[4933245776, 4933245775, 4933245774, 493324577...",True


Looks good! All we need to do now is keep just the columns we need, then save those for further processing.

In [16]:
places_of_worship[['id',
                   'tags.name','tags.denomination','tags.building',
                   'lat','lon','coords_imputed']].sort_values(by=['lon']).reset_index(drop=True).to_csv('places_of_worship.csv', index=False)

***